In [1]:
import pandas as pd
import re
import datetime as dt
import numpy as np

In [2]:
df = pd.read_json("ChatExport_2021-05-15/result.json")

In [3]:
def extract_date(x):
    return x['date']
df['date'] = df['messages'].apply(extract_date)

def extract_text(x):
    return x['text']
df['text'] = df['messages'].apply(extract_text)

df.drop(['id','name', 'type','messages'], axis=1, inplace=True)

In [4]:
def extract_zip(x):
    
    groups = re.search('(\d{6})', str(x)) 
    
    if groups != None:
        pin = groups.group()
        return pin
    else:
        return 0
    
df['pincode'] = df['text'].apply(extract_zip)
df = df[~(df['pincode']==0)]

In [5]:
df = df[(df['pincode'].str.startswith('56'))]
df['pincode'] = df['pincode'].astype('int')
df['date'] = df['date'].str.replace('T', ' ')

In [6]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
df.dtypes

date       datetime64[ns]
text               object
pincode             int32
dtype: object

In [7]:
def BBMP(x):
    try: 
        key = x.split('\n').index('BBMP')
    except: 
        pass 
    else: 
        return key

def Bangalore_Rural(x):
    try: 
        key = x.split('\n').index('Bangalore Rural')
    except: 
        pass 
    else: 
        return key

def Bangalore_Urban(x):
    try: 
        key = x.split('\n').index('Bangalore Urban')
    except: 
        pass 
    else: 
        return key

def test(x):   
        key_BBMP = BBMP(x)
        key_BR = Bangalore_Rural(x) 
        key_BU = Bangalore_Urban(x)
    
        if key_BBMP == key_BR == None:
            return key_BU
        elif key_BBMP == key_BU == None:
            return key_BR
        elif key_BR == key_BU == None:
            return key_BBMP

In [8]:
def check_str(x):

    if type(x) == str:
        return x.split('\n')[1].split('-')[0].split('1.')[1].strip()
   
    elif ''.join(x[0]).startswith('Vacc') and len(x)==2:
        return x[0].split('\n')[1].split('-')[0].split('1.')[1].strip()
    
    elif ''.join(x[0]).startswith('Vacc') and x[4].startswith(' at'):
        return x[4].split('at')[1].split(' on')[0].strip()    
    
    else:
        index = test(x[0])
        if type(index)==int:
            return x[0].split('\n')[index+1]
        else:
            return np.nan

df['Hospital'] = df['text'].apply(check_str)

In [12]:
df.to_excel('Hospital_vaccine.xlsx', index=False)